In [1]:
import pandas as pd

In [2]:
# get the constallation name abbreviations, to limit variable stars to sensible names
def load_const_file(const_file):
#const_file = 'constellation_names.txt'
    f_const = open(const_file)
    line = ' '
    const_abbr = []
    while line != '':
        line = f_const.readline()
        const_abbr.append(line.split('\t')[-1].replace('\n',''))
    const_abbr = const_abbr[2:]
    f_const.close()
    const_abbr_low = [ const.lower() for const in const_abbr ]
    return const_abbr_low

In [3]:
def load_seq_pattern(seq_file):
    '''
    get the sequence files available on BAA and return as regexp pattern

    '''
    seq = pd.read_csv('baa_sequences.txt',delimiter='\t')
    obj = seq['Object Id']
    display(obj)
    obj.describe()
    obs_lower = obj.str.lower()
    seq_pattern = '|'.join(obs_lower)
    return seq_pattern
    

In [4]:
in_file = '/home/john/astro/variable_star_data/baa/all_star_summary.csv'

In [5]:
# read in BAA variable star summary data as a Panda Data Frame
vsdb_full = pd.read_csv(in_file)
vsdb_full.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/john/astro/variable_star_data/baa/all_star_summary.csv'

In [ ]:
vsdb_full.describe()

In [ ]:
vsdb_full.hist('Number of Observations', bins=100)

In [ ]:
# can also call describe() on a Series object
vsdb_full['Maximum Magnitude'].describe()

In [ ]:
vsdb_full[['Variable Star', 'Number of Observations', 'Number of Observers']].sort_values(by='Number of Observations', ascending=False).head(30)

In [ ]:
# create a pattern to search which is an OR of each element in const_abbr (i.e. any constallation abbreviation)
const_abbrev = load_const_file('constellation_names.txt')
pattern = '|'.join(const_abbrev)
# need to watch the trailing OR, which seems to cause everything to match
pattern = pattern[:-1]
print(pattern)

In [ ]:
# I only want the stars where there are two fields in the name (e.g. 'TX Dra')
vsdb_allconst = vsdb_full
vsdb_allconst['Valid_nFields'] = vsdb_allconst['Variable Star'].str.count(' ')

#remove rows which don't have 1 nFields
vsdb_allconst.drop(vsdb_allconst[vsdb_allconst.Valid_nFields != 1].index, inplace=True)

# set the second field to be a 'Constellation' description.  Needs to be lower case to match pattern, above.
vs_const = vsdb_allconst['Variable Star'].str.split(expand=True)
vsdb_allconst['VS Constellation'] = vs_const[1].str.lower()

In [ ]:
# check whether str.contains is working...
vsdb_allconst['VS Constellation'].str.contains(pattern)==True

In [ ]:
# drop all rows where the conditional statement 
vsdb_allconst.drop(vsdb_allconst[vsdb_allconst['VS Constellation'].str.contains(pattern)==False].index, inplace=True)
vsdb_allconst.head()

In [ ]:
# vsdb, after initial filter on valid star name (two fields, contains a constellation abbreviation)
vsdb_allconst.describe()

In [ ]:
vsdb_allconst[['VS Constellation']].value_counts()

In [ ]:
# slicing via []
vsdb_allconst[vsdb_allconst['VS Constellation']=='cyg']

In [ ]:
#  this conditional statement returns a pandas Series of booleans
vsdb_allconst['VS Constellation']=='cyg'

In [ ]:
type(vsdb_allconst['VS Constellation']=='cyg')

In [ ]:
# check the dtype. Note the () are needed to prevent dtype operating on 'cyg'
(vsdb_allconst['VS Constellation']=='cyg').dtype

In [ ]:
# slicing via loc
# this is equivalent (I think!) to 
# vsdb[vsdb['VS Constellation']=='cyg']
vsdb_allconst.loc[vsdb_allconst['VS Constellation']=='cyg']

In [ ]:
vsdb_allconst[vsdb_allconst['VS Constellation']=='cyg'].sort_values(by='Number of Observations', ascending=False)

In [ ]:
# Having done that, now look at TARGET CONSTELLATIONS, but selecting the constallations that I 
# want to observe
# create a pattern to search which is an OR of each element in const_abbr (i.e. any constallation abbreviation)
vsdb_targets = vsdb_allconst
const_abbrev = load_const_file('constellation_names_targets.txt')
pattern = '|'.join(const_abbrev)
# need to watch the trailing OR, which seems to cause everything to match
pattern = pattern[:-1]
print(pattern)
# drop all rows where the conditional statement 
vsdb_targets.drop(vsdb_targets[vsdb_targets['VS Constellation'].str.contains(pattern)==False].index, inplace=True)
# start filtering on conditions
# first, range > v1.0
# this reduces the number of objects from ~2100 to ~1200
vsdb_targets = vsdb_targets[vsdb_targets['Range']>=1.0]
vsdb_targets.head()

In [ ]:
# start filtering on conditions
# first, range > v1.0
# this reduces the number of objects from ~2100 to ~1200
vsdb_targets = vsdb_targets[vsdb_targets['Range']>=1.0]
vsdb_targets.describe()

In [ ]:
# filter on min magnitude to find suitable binocular targets
#  reduces targets to ~200
vsdb_targets = vsdb_targets[vsdb_targets['Minimum Magnitude']<=9.0]
vsdb_targets.describe()

In [ ]:
vsdb_targets['Number of Observations'].hist()

In [ ]:
vsdb_targets.describe()

In [ ]:
# generate a target list from the 25% least observed variables
vs_target_list = vsdb_targets[vsdb_targets['Number of Observations']<=275]
# although probably better to try filtering on EXISTING baa sequence file first
# then possibly time since last obs?

In [ ]:
vs_target_list.drop(columns='Valid_nFields')

In [ ]:
vs_target_list.sort_values('VS Constellation')

In [ ]:
vs_target_list['VS Constellation'].value_counts()